In [4]:
import json
from braket.aws import AwsDevice
from braket.ocean_plugin import BraketSampler, BraketDWaveSampler

import numpy as np
import scipy.io
import matplotlib.pyplot as plt
import math
import time
import os
from os.path import join

import boto3
import io
import sys

from dwave.system.composites import EmbeddingComposite, FixedEmbeddingComposite

**Note**: Enter the S3 bucket and device below.

In [5]:
# Enter the S3 bucket you created during onboarding in the code below
my_bucket = "amazon-braket-wugroup-us-east-1" # the name of the bucket
my_prefix = "jiaqileng/qubo" # the name of the folder in the bucket
s3_folder = (my_bucket, my_prefix)
client = boto3.client('s3')

In [6]:
# session and device
#device_name = "arn:aws:braket:::device/qpu/d-wave/Advantage_system4"
device_name = "arn:aws:braket:us-west-2::device/qpu/d-wave/Advantage_system6"
device = AwsDevice(device_name)
print('Device:', device)

Device: Device('name': Advantage_system6.1, 'arn': arn:aws:braket:us-west-2::device/qpu/d-wave/Advantage_system6)


# Problem Setting: Quadratic Programming

We consider the following quadratic programming (QP) problem: $$\min \frac{1}{2} x^T Q x + b^Tx,$$
where the variable $x$ is restricted to the unit square $\{-1 \le x_j \le 1\}$, and subject to the equality constraint $Q_c x = b_c$.

When $Q$ is indefinite (i.e., it has negative eigenvalues), this problem is non-convex.

### Quantum algorithms for QP problems

We have two quantum algorithms available for use: (1) the standard Quantum Adiabatic Algorithm (QAA); (2) Quantum Hamiltonian Descent (QHD). 

### Anneal schedules in QAA & QHD

On D-Wave, the anneal schedule can be set via user-specified solver parameters. We use the following two anneal schedules for quantum algorithms: 
1. **schedule A**: the default anneal schedule on D-Wave: "[[0,0],[20,1]"
2. **schedule B**: a prolonged anneal schedule with anneal time T = 800 and two quenches "[[0,0],[400, 0.3],[640, 0.6],[800,1]]".This is the schedule for QHD only. 



In [7]:
# import data directory
sys.path.insert(1, '../../../')
from config import * 

# Benchmark information
dimension = 75
sparsity = 5
benchmark_name = f"QP-{dimension}d-{sparsity}s"
benchmark_dir = join(DATA_DIR_QP, benchmark_name)
num_instances = 50

In [8]:
def list_to_vec(sample_list, P):
    # sample_list = a sample from the response of the D-Wave sampler
    # P = the full precision matrix for d-dim vector
    list_len = len(sample_list)
    binary_vec = np.empty((list_len))
    for k in np.arange(list_len):
        binary_vec[k] = sample_list['%s'%k]
        
    return P @ binary_vec

def get_qubo(Q, b, Q_c, b_c, H, sigma):
    # We assume all continuous variables are between 0 and 1.
    
    Q_qhd = H.T @ (Q / 2 + sigma * Q_c.T @ Q_c) @ H
    R_qhd = (b - 2 * sigma * b_c.T @ Q_c) @ H

    # Build QUBO dict
    QM = dict()
    for i in range(len(H[0])):
        if (np.abs(Q_qhd[i,i] + R_qhd[i]) > 0):
            QM[str(i),str(i)] = Q_qhd[i,i] + R_qhd[i]
        for j in np.arange(i+1, len(H[0])):
            if (np.abs(Q_qhd[i,j]) > 0):
                QM[str(i),str(j)] = 2 * Q_qhd[i,j]

    # maximum strength
    max_interaction_qhd = np.max(np.abs(np.array(list(QM.values()))))
    
    return QM, max_interaction_qhd

def get_delay_list(Q_c, dimension, resolution, embedding):
    var_equality_constraint = []
    for k in range(len(Q_c)):
        for j in range(dimension):
            if Q_c[k][j] != 0:
                var_equality_constraint.append(j)
    var_equality_constraint = np.unique(var_equality_constraint)

    qubit_list = []
    for var in var_equality_constraint:
        var_to_qbt = np.arange(var*resolution, (var+1)*resolution)
        qubit_list.append(var_to_qbt)
    qubit_list = np.unique(qubit_list)

    delay_list = []
    for qubit in qubit_list:
        physical_qbt = embedding[str(qubit)]
        for j in physical_qbt:
            delay_list.append(j)
    return delay_list

In [12]:
# Quantum Adiabatic Algorithm (QAA)
# Use the Radix-2 encoding of real numbers

print('Device:', device)

# QUBO parameters
sigma = 0 # penalty coefficient, set as 100 if equality_constraint != 0
resolution = 8
p = np.array([2**(-(int(np.log2(resolution))-i+1)) for i in range(int(np.log2(resolution))+1)])
p[0] = 2**(-(int(np.log2(resolution))))
Id = np.identity(dimension)
P = np.kron(Id, p) # Precision vector for Radix-2 encoding

# QPU parameters
numruns = 1000
USE_EXISTING_EMBEDDING = True
embedding_filename = join(benchmark_dir, f"config/advantage6_qaa_rez{resolution}_embedding.json")
if USE_EXISTING_EMBEDDING:
    # Load Embedding from json
    with open(embedding_filename, 'r', encoding='utf-8') as f:
        embedding = json.loads(f.read())
    
    #js_content = open(embedding_filename)
    #embedding = json.loads(js_content)
    '''
    # Use the following for S3 object
    s3object = client.get_object(Bucket=my_bucket, Key=embedding_filename)
    js_content = s3object['Body'].read().decode('UTF-8')
    embedding = json.loads(js_content)
    '''

SCHEDULE = "A"
if SCHEDULE == "A":
    schedule = [[0,0],[20,1]] # the default anneal schedule
elif SCHEDULE == "B":
    schedule = [[0,0],[400, 0.3],[640, 0.6],[800,1]] # as a comparison to QHD
else:
    raise ValueError("Anneal schedule not recognized.")


for instance in range(num_instances):
    print(f"Run standard QAA (Radix-2) on instance No. {instance} from {benchmark_name}.")
    
    # Load instance data from S3
    instance_filename = join(benchmark_dir, f"instance_{instance}/instance_{instance}.npy")
    with open(instance_filename, 'rb') as f:
        Q = np.load(f)
        b = np.load(f)
        Q_c = np.load(f)
        b_c = np.load(f)
    
    # Build QUBO dict
    qaa_samples = np.zeros(shape=(numruns, dimension))
    QM, max_interaction = get_qubo(Q, b, Q_c, b_c, P, sigma)
    chainstrength = 1.1 * max_interaction
    
    if (USE_EXISTING_EMBEDDING == False) & (instance == 0):
        # Generate a new embedding and save it to {benchmark_name}/config
        qpu = BraketDWaveSampler(s3_folder,device_name)
        sampler = EmbeddingComposite(qpu)
        response_qaa = sampler.sample_qubo(QM, 
                                    chain_strength=chainstrength, 
                                    num_reads=numruns,
                                    anneal_schedule=schedule,
                                    return_embedding=True)
        embedding = response_qaa.info["embedding_context"]["embedding"]
        with open(embedding_filename, "w") as outfile:
            json.dump(embedding, outfile)
        print(f"Minor embedding saved.")
    else: 
        qpu = BraketDWaveSampler(s3_folder,device_name)
        sampler = FixedEmbeddingComposite(qpu, embedding)
        response_qaa = sampler.sample_qubo(QM, 
                                    chain_strength=chainstrength, 
                                    num_reads=numruns,
                                    anneal_schedule=schedule)
    
    samples_list_qaa = list(response_qaa.samples())
    for i in range(numruns):
        qaa_samples[i] = list_to_vec(samples_list_qaa[i], P)
    
    # Save QAA samples
    qaa_filename = join(benchmark_dir, f"instance_{instance}/advantage6_qaa_schedule{SCHEDULE}_rez{resolution}_sample_{instance}.npy")
    with open(qaa_filename, 'wb') as f:
        np.save(f, qaa_samples)
    print(f"D-Wave standard QAA (schedule {SCHEDULE}): data saved.")
    
    # Finish processing instance 
    print(f"Instance No. {instance} from {benchmark_name}: completed.\n")

Device: Device('name': Advantage_system6.1, 'arn': arn:aws:braket:us-west-2::device/qpu/d-wave/Advantage_system6)
Run standard QAA (Radix-2) on instance No. 0 from QP-75d-5s.


AttributeError: 'AwsDevice' object has no attribute 'aws_session'

In [13]:
# Quantum Hamiltonian Descent (QHD)
# Use the Hamming-weight encoding of real numbers

print('Device:', device)

# QUBO parameters
sigma = 0 # penalty coefficient, set as 100 if equality_constraint != 0
resolution = 8
h = np.ones(resolution) / resolution
Id = np.identity(dimension)
H = np.kron(Id, h) # Precision vector for QHD

# QPU parameters
numruns = 1000
USE_EXISTING_EMBEDDING = True
embedding_filename = join(benchmark_dir, f"config/advantage6_qhd_rez{resolution}_embedding.json")
if USE_EXISTING_EMBEDDING:
    # Load Embedding from json
    with open(embedding_filename, 'r', encoding='utf-8') as f:
        embedding = json.loads(f.read())
        
    '''
    # Use the following for S3 object
    s3object = client.get_object(Bucket=my_bucket, Key=embedding_filename)
    js_content = s3object['Body'].read().decode('UTF-8')
    embedding = json.loads(js_content)
    '''
    
schedule = [[0,0],[400, 0.3],[640, 0.6],[800,1]] # schedule B


for instance in range(num_instances):
    print(f"Run QHD on instance No. {instance} from {benchmark_name}.")
    
    # Load instance data from S3
    instance_filename = join(benchmark_dir, f"instance_{instance}/instance_{instance}.npy")
    with open(instance_filename, 'rb') as f:
        Q = np.load(f)
        b = np.load(f)
        Q_c = np.load(f)
        b_c = np.load(f)
    
    
    # Build QUBO dict
    qhd_samples = np.zeros(shape=(numruns, dimension))
    QM, max_interaction = get_qubo(Q, b, Q_c, b_c, H, sigma)
    chainstrength = 1.1 * max_interaction
    
    if (USE_EXISTING_EMBEDDING == False) & (instance == 0):
        # Generate a new embedding and save it to {benchmark_name}/config
        qpu = BraketDWaveSampler(s3_folder,device_name)
        sampler = EmbeddingComposite(qpu)
        response_qhd = sampler.sample_qubo(QM, 
                                    chain_strength=chainstrength, 
                                    num_reads=numruns,
                                    anneal_schedule=schedule,
                                    return_embedding=True)
        embedding = response_qhd.info["embedding_context"]["embedding"]
        with open(embedding_filename, "w") as outfile:
            json.dump(embedding, outfile)
        print(f"Minor embedding saved.")
    else: 
        qpu = BraketDWaveSampler(s3_folder,device_name)
        sampler = FixedEmbeddingComposite(qpu, embedding)
        response_qhd = sampler.sample_qubo(QM, 
                                    chain_strength=chainstrength, 
                                    num_reads=numruns,
                                    anneal_schedule=schedule)
    
    samples_list_qhd = list(response_qhd.samples())
    for i in range(numruns):
        qhd_samples[i] = list_to_vec(samples_list_qhd[i], H)
    
    # Save QHD samples
    qhd_filename = join(benchmark_dir, f"instance_{instance}/advantage6_qhd_rez{resolution}_sample_{instance}.npy")
    with open(qhd_filename, 'wb') as f:
        np.save(f, qhd_samples)
    print(f"D-Wave QHD: data saved.")
    
    # Finish processing instance 
    print(f"Instance No. {instance} from {benchmark_name}: completed.\n")

Device: Device('name': Advantage_system6.1, 'arn': arn:aws:braket:us-west-2::device/qpu/d-wave/Advantage_system6)
Run QHD on instance No. 0 from QP-75d-5s.


AttributeError: 'AwsDevice' object has no attribute 'aws_session'